In [47]:
import math
from collections import Counter
import numpy as np

In [48]:

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
MAX_DEPTH = 2

In [49]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )


In [50]:
import random

from itertools import product

def unique_moves(board, player):
    #cheks for only moves
    for move in valid_moves(board):
        cp_board= np.copy(board)
        play(cp_board,move,player)
        if four_in_a_row(cp_board, player):
            return move
    for move in valid_moves(board):
        cp_board= np.copy(board)
        play(cp_board,move,-player)
        if four_in_a_row(cp_board, -player):
            return move
    return -1
def calculate_board_hash (board):
    #used for Look up table
    cnt =""
    for i,j in product(range(NUM_COLUMNS), range(COLUMN_HEIGHT)):
        if board[i][j]==1:
          cnt+="2"
        else:
            if board[i][j]==-1:
                cnt+="1"
            else: cnt+="0"
    return cnt

def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))

    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    #minmax wrapper
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        d = {'key': 'value'}
        bv = float('nan')
        return minmax(np.copy(board), player,0,bv,d)


def minmax(board, player, depth,best_value, lut):
    #return value of minmax contains the move as well as the evaluation of the position
    if calculate_board_hash(board) in lut:
        #if find in look up table return
        return  -1,lut[calculate_board_hash(board)]
    if four_in_a_row(board, -player):
        #if last player won return -player
        lut[calculate_board_hash(board)] =-player
        return -1, -player
    possible_moves = valid_moves(board)
    if not possible_moves:
        #if no moves are possible return 0
        lut[calculate_board_hash(board)] =0
        return -1,0
    depth+=1;
    first_eval = 1
    best_eval =float('NaN')
    ret_val=-1
    only_move = unique_moves(board, player)
    if only_move!=-1:
        #if one move wins or only one move prevenst loss, play that move and analyze deeper (depth--)
        recur_board= np.copy(board)
        play(recur_board, only_move, player)
        _,values= minmax(np.copy(recur_board), -player,depth-1,best_eval,lut)
        lut[calculate_board_hash(board)] =values
        return only_move,values
    if depth > MAX_DEPTH:
        #if depth is too high procede randomly
        return  -1,montecarlo(np.copy(board), -player)
    for val in possible_moves:
        #try all possible moves
         recur_board= np.copy(board)
         play(recur_board, val, player)

         if first_eval:
             #if it's the first move you try don't look for alpha-beta pruning
             first_eval =0
             _,values=  minmax(np.copy(recur_board), -player,depth,best_eval,lut)
             best_eval= values*player
             ret_val=val
         else:
             _,values= minmax(np.copy(recur_board), -player,depth,best_eval,lut)
             if best_eval<values*player:
                 ret_val=val
             best_eval=max(best_eval,values*player)
             if not math.isnan(best_value) and best_eval>-best_value:
                 #if the move you just tried makes the branch uninteresting return (alpha-beta pruning)
                lut[calculate_board_hash(board)] =best_eval*player
                return val,best_eval*player
    lut[calculate_board_hash(board)] =best_eval
    return ret_val,best_eval*player


class Node():
    #data structure for MCTS
    def __init__(self, board, parent = None):
        self.visits = 1
        self.reward = 0.0
        self.board = board
        self.children = []
        self.children_move = []
        self.parent = parent


def MCTS(maxIter, root, player0, factor):
    for inter in range(maxIter):
        front, player = treePolicy( root , player0 , factor )
        reward = _mc(np.copy(front.board), player)
        backup(front,reward,player)
    ans = bestChild(root,0)
    return ans
def treePolicy( node, player , factor ):
    #analyze the tree
	while len(valid_moves(node.board))>0 and four_in_a_row(board,player) == 0:
		if  len(valid_moves(node.board)) >len(node.children) :
			return expand(node, player), -player
		else:
			node = bestChild ( node , factor )
			player *= -1
	return node, player

def expand( node, player ):
    #expands the node in the tree
	tried_children_move = [m for m in node.children_move]
	possible_moves = valid_moves(node.board)

	for move in possible_moves:
		if move not in tried_children_move:
			new_state =  Node(np.copy(node.board),node)
			play(new_state.board, move, player)
			break

	node.children.append(new_state)

	node.children_move.append(move)

	return new_state

def bestChild(node,factor):
    #selects the best children/one of the best
    bestscore = -10000000.0

    bestChildren = []
    for c in node.children:
        exploit = c.reward / c.visits
        explore = math.sqrt(math.log(2.0*node.visits)/float(c.visits))
        score = exploit + factor*explore
        if score == bestscore:
            bestChildren.append(c)
        if score > bestscore:
            bestChildren = [c]
            bestscore = score

    return random.choice(bestChildren)



def backup( node , reward, turn ):
    #back propagates the result of the currently analized node up the tree using an avarage of the results
    #calculated using the numebr of visits
	while node != None:
		node.visits += 1
		node.reward -= turn*reward
		node = node.parent
		turn *= -1
	return

def print_board(board):
    print("[",end ="")
    for i in range (NUM_COLUMNS):
        print("\n")
        print(i,end="")
        for j in range(COLUMN_HEIGHT):
            if board[i][j]==0:
                print("-",end="")
            if board[i][j]==1:
                print("X",end="")
            if board[i][j]==-1:
                print("O",end="")
    print("]")

In [51]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board,3,1)
play(board,0,-1)
play(board,4,1)
play(board,1,-1)
play(board,2,1)





print(board)

player =1#to start set player as the last player who moved
move = 0
game= Node(board)
factor =0.5
while not four_in_a_row(game.board, player)and len(valid_moves(game.board))!=0:
    player*=-1
    if player ==1:
        game= MCTS(1000,game,player,factor)
    else:
        move, _ = eval_board(game.board, player)
        play(game.board,move,player)
        game = Node(game.board)
    print_board(game.board)
    print("Move player by player: ", end="")
    if player==1: print("MCTS (X)")
    else: print("MinMax (O)")
if four_in_a_row(game.board, player):
    print("The match was won by player: ")
    if player ==1:
        print ("X")
    else:
        print("O")
else:
    if(len(valid_moves(game.board))==0):
        print("Draw")





[[-1  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


KeyboardInterrupt: 